# Final Masters Project

## Name: Sreekanth Palagiri, Student ID: R00184198

## Project Topic: Evaluation of Ensemble Approach for Sentiment Analysis on a Small Dataset

##NoteBook1: Trainer Flair


### **Mount google drive**

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls "gdrive/My Drive/Colab Notebooks/Masters Project"

'Airline Tweets dataset'  'Sentence Polarity Dataset'
 glove.6B.300d.txt	   VMDataset


### **Install Flair**

In [ ]:
!pip install flair

     |████████████████████████████████| 286kB 12.3MB/s 
     |████████████████████████████████| 798kB 20.5MB/s 
     |████████████████████████████████| 983kB 37.9MB/s 
     |████████████████████████████████| 19.7MB 7.3MB/s 
     |████████████████████████████████| 1.2MB 40.0MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 2.1MB 37.7MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 38.3MB/s 
     |████████████████████████████████| 901kB 33.2MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=d0c3058e2bd8fb3a8111375126f401d7c500ef9ed0b6b8a5f6d7313596360993
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for mpld3: filename

### **Load Data and Preprocess**

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/airlinecomplaint.csv")
print(df.groupby(['label']).size())
df.head()

label
0    1700
1    1700
dtype: int64


,tweet,label
0,@united UA maintenance issues strike again. Fl...,0
1,With @AirCanada taking away more @ACAltitude b...,1
2,@DudePerfect @AmericanAir that's a lot of miss...,1
3,Oh @AmericanAir we promise to try hard not to ...,1
4,@AmericanAir liked to me that I couldn't chang...,0


**Preprocessor to Remove all special characters except emoticons**

In [ ]:
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[^A-Za-z0-9\']+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text

print(df['tweet'][19])
print(preprocessor(df['tweet'][19]))

So @AmericanAir I'm going to need you all to get this baggage thing together. First a lost bag, then a list carousel. Yeah, umm...
so americanair i'm going to need you all to get this baggage thing together first a lost bag then a list carousel yeah umm 


In [ ]:
df['tweet'] = df['tweet'].apply(preprocessor)

In [ ]:
df1=df.copy()

In [ ]:
df['label'][df.label==0]='Negative'
df['label'][df.label==1]='Positive'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test, sentiment_train, sentiment_test = train_test_split(df1['tweet'], df1['label'], 
                                                                      random_state=1, test_size=0.15, 
                                                                      shuffle=False)


print('Length of train set:',len(df_train),'Length of test set:',len(df_test))

Length of train set: 2890 Length of test set: 510


**Label Encoding to the format specified by Flair**

In [ ]:
df['label'] = '__label__' + df['label'].astype(str)
df=df[['label','tweet']]
df.head()

,label,tweet
0,__label__Negative,united ua maintenance issues strike again flt...
1,__label__Positive,with aircanada taking away more acaltitude ben...
2,__label__Positive,dudeperfect americanair that's a lot of missi...
3,__label__Positive,oh americanair we promise to try hard not to c...
4,__label__Negative,americanair liked to me that i couldn't chang...


**Flair Needs validation, test and train data in csv's.**

**Creating csv files with 85-10-5 split.**

In [ ]:
df.iloc[0:int(len(df)*0.85)].to_csv('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/train.csv', sep='\t', 
                                                index = False, header = False)

df.iloc[int(len(df)*0.85):int(len(df)*0.95)].to_csv('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/test.csv', sep='\t', 
                                                                      index = False, header = False)

df.iloc[int(len(df)*0.95):].to_csv('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/val.csv', sep='\t', 
                                               index = False, header = False);

**Creating corpus with Train, Test and Validation data for model training.**

In [ ]:
from flair.data import Corpus
from flair.datasets import ClassificationCorpus
#from flair.data import Sentence

data_folder = '/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/'

# this is the folder in which training, test and dev files reside

corpus: Corpus = ClassificationCorpus(data_folder,
                                      test_file='test.csv',
                                      dev_file='val.csv',
                                      train_file='train.csv')

2021-04-29 13:42:17,003 Reading data from /content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models
2021-04-29 13:42:17,004 Train: /content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/train.csv
2021-04-29 13:42:17,010 Dev: /content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/val.csv
2021-04-29 13:42:17,013 Test: /content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/test.csv


### **Traing Flair Model**


In [ ]:
from torch.optim.adam import Adam
from flair.data import Corpus
#from flair.datasets import TREC_6
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer


# 2. create the label dictionary
label_dict = corpus.make_label_dictionary()

# 3. initialize transformer document embeddings (many models are available)
document_embeddings = TransformerDocumentEmbeddings('distilbert-base-uncased', fine_tune=True)

# 4. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)

# 5. initialize the text classifier trainer with Adam optimizer
trainer = ModelTrainer(classifier, corpus, optimizer=Adam)

# 6. start the training
trainer.train('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/resources/taggers/trec',
              learning_rate=3e-5, # use very small learning rate
              mini_batch_size=16,
              mini_batch_chunk_size=4, # optionally set this if transformer is too much for your machine
              max_epochs=10, # terminate after 5 epochs
              )

2021-04-29 13:42:17,087 Computing label dictionary. Progress:


100%|██████████| 3230/3230 [00:02<00:00, 1178.53it/s]

2021-04-29 13:42:19,971 [b'Negative', b'Positive']



2021-04-29 13:42:36,603 ----------------------------------------------------------------------------------------------------
2021-04-29 13:42:36,610 Model: "TextClassifier(
  (document_embeddings): TransformerDocumentEmbeddings(
    (model): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(i

{'dev_loss_history': [0.5164310336112976,
  0.5258457660675049,
  0.803408145904541,
  1.4712188243865967,
  1.67872154712677,
  2.0522310733795166,
  1.8923404216766357,
  1.9267995357513428,
  2.016023874282837,
  1.9591660499572754],
 'dev_score_history': [0.7353,
  0.7529,
  0.7765,
  0.7882,
  0.7588,
  0.7471,
  0.7941,
  0.7824,
  0.7765,
  0.7824],
 'test_score': 0.7824,
 'train_loss_history': [0.5418047452170546,
  0.30314308501142,
  0.1698804525608642,
  0.08725928794009738,
  0.032958351460072824,
  0.01764266986120867,
  0.04187967008737704,
  0.03484624709201103,
  0.04130343543434504,
  0.02310967315551803]}

In [ ]:
from flair.data import Sentence

model_flair=TextClassifier.load('/content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/resources/taggers/trec/best-model.pt')

results=[]
Y_pred=[]
for i in df_test.index:
    sentence=Sentence(df_test[i])
    model_flair.predict(sentence)
    if sentence.get_labels()[0].value=='Positive':
      score=1-sentence.get_labels()[0].score
      Y_pred.append([1])
    else:
      score=sentence.get_labels()[0].score
      Y_pred.append([0])
    results.append([score,1-score])

2021-04-29 14:02:28,623 loading file /content/gdrive/My Drive/Colab Notebooks/Masters Project/Airline Tweets dataset/Models/resources/taggers/trec/best-model.pt


In [ ]:
from sklearn import metrics

Y_test=sentiment_test
print('F1 Score:',metrics.f1_score(Y_test,Y_pred),
      'Precision:',metrics.precision_score(Y_test,Y_pred),
      'Recall:',metrics.recall_score(Y_test,Y_pred),
      'Accuracy:',metrics.accuracy_score(Y_test,Y_pred))

F1 Score: 0.7891682785299806 Precision: 0.8095238095238095 Recall: 0.769811320754717 Accuracy: 0.7862745098039216


In [ ]:
print(metrics.confusion_matrix(Y_test, Y_pred))

[[197  48]
 [ 61 204]]
